In [1]:
# STEP 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# STEP 2: Install Required Libraries
!pip install transformers datasets scikit-learn gradio -q


In [3]:
# STEP 3: Load the AG News CSV files
import pandas as pd

# Change the paths if your folder is different
train_df = pd.read_csv("/content/drive/MyDrive/ag-news dtset/train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/ag-news dtset/test.csv")

# Rename columns if necessary
train_df.columns = ["label", "title", "description"]
test_df.columns = ["label", "title", "description"]

# We'll use 'title' for classification
train_df["text"] = train_df["title"]
test_df["text"] = test_df["title"]


In [4]:
# STEP 4: Sample the training data for faster training
train_df = train_df.sample(n=5000, random_state=42).reset_index(drop=True)


In [5]:
# STEP 5: Encode labels
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_df["label"] = label_encoder.fit_transform(train_df["label"])
test_df["label"] = label_encoder.transform(test_df["label"])


In [6]:
# STEP 6: Tokenize with Hugging Face BERT tokenizer
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

train_encodings = tokenizer(list(train_df["text"]), truncation=True, padding=True)
test_encodings = tokenizer(list(test_df["text"]), truncation=True, padding=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [7]:
# STEP 7: Create Torch Datasets
import torch

class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = NewsDataset(train_encodings, list(train_df["label"]))
test_dataset = NewsDataset(test_encodings, list(test_df["label"]))


In [8]:
# STEP 8: Load Pretrained BERT Model
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# STEP 9: Define Evaluation Metrics
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    return {"accuracy": acc, "f1": f1}


In [10]:
# STEP 10: Training
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

trainer.train()

/tmp/ipython-input-10-2456476534.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mohammadbilalhassan68 (mohammadbilalhassan68-islamia-college-peshawar) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.589700


TrainOutput(global_step=939, training_loss=0.4173130851964981, metrics={'train_runtime': 7904.1543, 'train_samples_per_second': 1.898, 'train_steps_per_second': 0.119, 'total_flos': 316047274920000.0, 'train_loss': 0.4173130851964981, 'epoch': 3.0})

In [11]:
# STEP 11: Evaluate
trainer.evaluate()


{'eval_loss': 0.5715417861938477,
 'eval_accuracy': 0.8556578947368421,
 'eval_f1': 0.8556960063932154,
 'eval_runtime': 1076.2775,
 'eval_samples_per_second': 7.061,
 'eval_steps_per_second': 0.111,
 'epoch': 3.0}

In [12]:
# STEP 12: Deploy with Gradio
import gradio as gr

def classify_news(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    return label_encoder.inverse_transform([prediction])[0]

gr.Interface(fn=classify_news, inputs="text", outputs="text", title="AG News Classifier").launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3f9d8b733782656c57.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
